In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import Counter
from operator import itemgetter
from itertools import tee
import pysam
import numpy as np
import cPickle as pickle
import matplotlib.pyplot as plt

In [3]:
from pyUMI import umi_stats as stats
from pyUMI import multimap_resolve_lib as mr
from pyUMI import util

In [4]:
import pyUMI.umiDistAnalyzer as dst

In [5]:
import logging
logging.basicConfig(level=logging.INFO)

In [6]:
st = pysam.AlignmentFile("/data/UMI/data/MUS/sample_AAAACT.bam","rb")

In [22]:
md = pickle.load(open('/data/UMI/sample_AAAACT_md_umi.pkl','r'))

In [7]:
ud = pickle.load(open('/data/UMI/sample_AAAACT_ud_umi.pkl','r'))

In [25]:
#sum(md['TTTTTTTTTT'].values())

In [20]:
#uregions = [ref for ref,start in dst.region_based_generator(ud['TTTTTTTTTT'])]


In [38]:
mregions = [ref for ref,start in dst.region_based_generator(multi_read_dist_table, multi_read=True)]

In [40]:
uregions = [ref for ref,start in dst.region_based_generator(unique_read_dist_table)]

In [42]:
st.reset()
t = dst.get_umi_list(st,include_unmapped=True)
t2 = dst.get_umi_list(st)

In [44]:
print len(t)
print len(t2)

545925
515393


--------------------------------

In [21]:
%%time
g1 = field_based_generator(st, 'flag', [4], False)
g2 = tag_based_generator(g1, 'XM', 'TTTTTTTTTT')
total = len([i for i in g2])
print total

3974
CPU times: user 25.1 s, sys: 150 ms, total: 25.2 s
Wall time: 25.3 s


In [22]:
%%time
g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
g2 = field_based_generator(g1, 'flag', [4], False)
total = len([i for i in g2])
print total

3974
CPU times: user 22.3 s, sys: 856 ms, total: 23.1 s
Wall time: 23.2 s


In [24]:
%%time
g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
g2 = field_based_generator(g1, 'flag', [4], False)
g3 = tag_based_generator(g2, 'NH', [1])
uniques = len([i for i in g3])
print uniques

681
CPU times: user 22 s, sys: 129 ms, total: 22.1 s
Wall time: 22.1 s


In [25]:
%%time
g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
g2 = field_based_generator(g1, 'flag', [4], False)
g3 = tag_based_generator(g2, 'NH', [1], False)
multi_reads = len([i for i in g3])
print multi_reads

3293
CPU times: user 22.2 s, sys: 131 ms, total: 22.3 s
Wall time: 22.4 s


In [28]:
%%time
g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
g2 = field_based_generator(g1, 'flag', [4], False)
refs = Counter([item.reference_name for item in g2])
print len(refs)

24
CPU times: user 21.7 s, sys: 121 ms, total: 21.8 s
Wall time: 21.8 s


In [56]:
%%time
g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
g2 = field_based_generator(g1, 'flag', [4], False)
g3 = tag_based_generator(g2, 'NH', [1])
urefs = Counter([item.reference_name for item in g3])
print len(urefs)

20
CPU times: user 21.8 s, sys: 125 ms, total: 21.9 s
Wall time: 22 s


In [30]:
%%time
g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
g2 = field_based_generator(g1, 'flag', [4], False)
g3 = tag_based_generator(g2, 'NH', [1], False)
mrefs = Counter([item.reference_name for item in g3])
print len(mrefs)

24
CPU times: user 21.6 s, sys: 135 ms, total: 21.7 s
Wall time: 21.7 s


---------------------

In [57]:
%%time
Coordinate_range = {}
for ref in urefs.keys():
    g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
    g2 = field_based_generator(g1, 'flag', [4], False)
    g3 = tag_based_generator(g2, 'NH', [1])
    g4 = field_based_generator(g3, 'reference_name', [ref])
    coordinates = [item.reference_start for item in g4]
    Coordinate_range.update({ref:np.unique(coordinates)})

CPU times: user 7min 14s, sys: 2.65 s, total: 7min 17s
Wall time: 7min 17s


In [59]:
co_range = []
for k in Coordinate_range.keys():
    co_range.append((k, max(Coordinate_range[k]) - min(Coordinate_range[k])))
co_range

[('chr7', 138250599),
 ('chr6', 79340217),
 ('chr5', 139874219),
 ('chr4', 141466302),
 ('chr3', 96044020),
 ('chr2', 160621156),
 ('chr1', 176949189),
 ('chr9', 89284770),
 ('chr8', 109159510),
 ('chr13', 1),
 ('chr12', 75531664),
 ('chr11', 114433609),
 ('chr10', 41564404),
 ('chr17', 74521802),
 ('chr16', 0),
 ('chr15', 69041777),
 ('chr14', 90964777),
 ('chr19', 31566992),
 ('chr18', 28601872),
 ('chrX', 138917645)]

In [61]:
from math import log

In [69]:
from pyUMI import mm10

In [70]:
mm10.ref_lengths

{'chr': 195471971,
 'chr10': 130694993,
 'chr11': 122082543,
 'chr12': 120129022,
 'chr13': 120421639,
 'chr14': 124902244,
 'chr15': 104043685,
 'chr16': 98207768,
 'chr17': 94987271,
 'chr18': 90702639,
 'chr19': 61431566,
 'chr1_GL456210_random': 169725,
 'chr1_GL456211_random': 241735,
 'chr1_GL456212_random': 153618,
 'chr1_GL456213_random': 39340,
 'chr1_GL456221_random': 206961,
 'chr2': 182113224,
 'chr3': 160039680,
 'chr4': 156508116,
 'chr4_GL456216_random': 66673,
 'chr4_GL456350_random': 227966,
 'chr4_JH584292_random': 14945,
 'chr4_JH584293_random': 207968,
 'chr4_JH584294_random': 191905,
 'chr4_JH584295_random': 1976,
 'chr5': 151834684,
 'chr5_GL456354_random': 195993,
 'chr5_JH584296_random': 199368,
 'chr5_JH584297_random': 205776,
 'chr5_JH584298_random': 184189,
 'chr5_JH584299_random': 953012,
 'chr6': 149736546,
 'chr7': 145441459,
 'chr7_GL456219_random': 175968,
 'chr8': 129401213,
 'chr9': 124595110,
 'chrM': 16299,
 'chrUn_GL456239': 40056,
 'chrUn_GL456359'

In [71]:
a ={'chr': 195471971,
 'chr10': 130694993,
 'chr11': 122082543,
 'chr12': 120129022,
 'chr13': 120421639,
 'chr14': 124902244,
 'chr15': 104043685,
 'chr16': 98207768,
 'chr17': 94987271,
 'chr18': 90702639,
 'chr19': 61431566,
 'chr1_GL456210_random': 169725,
 'chr1_GL456211_random': 241735,
 'chr1_GL456212_random': 153618,
 'chr1_GL456213_random': 39340,
 'chr1_GL456221_random': 206961,
 'chr2': 182113224,
 'chr3': 160039680,
 'chr4': 156508116,
 'chr4_GL456216_random': 66673,
 'chr4_GL456350_random': 227966,
 'chr4_JH584292_random': 14945,
 'chr4_JH584293_random': 207968,
 'chr4_JH584294_random': 191905,
 'chr4_JH584295_random': 1976,
 'chr5': 151834684,
 'chr5_GL456354_random': 195993,
 'chr5_JH584296_random': 199368,
 'chr5_JH584297_random': 205776,
 'chr5_JH584298_random': 184189,
 'chr5_JH584299_random': 953012,
 'chr6': 149736546,
 'chr7': 145441459,
 'chr7_GL456219_random': 175968,
 'chr8': 129401213,
 'chr9': 124595110,
 'chrM': 16299,
 'chrUn_GL456239': 40056,
 'chrUn_GL456359': 22974,
 'chrUn_GL456360': 31704,
 'chrX': 171031299,
 'chrY': 91744698}



In [10]:
!pwd

/data/UMI


In [25]:
%%time
!samtools view -F 0x04 /data/UMI/data/MUS/sample_AAAACT.bam | grep -w 'NH:i:1' | grep -w 'XM:Z:TTTTTTTTTT' | wc -l

681
CPU times: user 596 ms, sys: 100 ms, total: 696 ms
Wall time: 32.9 s


In [26]:
g1 = tag_based_generator(st, 'XM', 'TTTTTTTTTT')
g2 = field_based_generator(g1, 'flag', [4], False)
g3 = tag_based_generator(g2, 'NH', [1])

In [27]:
%%time
len([i for i in g3])

CPU times: user 22 s, sys: 120 ms, total: 22.1 s
Wall time: 22.2 s


681

In [28]:
import pickle

In [29]:
diffs = pickle.load(open('/data/UMI/Optimization/umiDiffs.pkl','r'))

In [30]:
diffs

{'AAATTACACG': 0,
 'ATTAATAGAT': 26,
 'GATGACATGA': 19,
 'TTATTTATTA': 57,
 'TTATTTATTC': 4,
 'TTATTTATTG': 0}

In [31]:
g1 = field_based_generator(st, 'flag', [4], False)
u = [r.get_tag('XM') for r in g1]
umi_list = Counter(u).keys()

In [32]:
umi_list

['AAATTACACG',
 'TTATTTATTG',
 'TTATTTATTC',
 'GATGACATGA',
 'TTATTTATTA',
 'ATTAATAGAT',
 'ATGACTTATT',
 'CGCCGTACTT',
 'ATATAAAGGC',
 'TGCCATGACA',
 'TTATTTATTT',
 'GATGACATGT',
 'ATAAATCACA',
 'ATAAACCGTA',
 'CGACCACACT',
 'TTATTTTACG',
 'ATTAATAGAG',
 'TTATTTTACA',
 'TATTTCTGGT',
 'ATAAACCGTG',
 'ATTAATAGAC',
 'TGTCACGAAA',
 'GTGCCGGAGG',
 'TCGGGTGCAA',
 'TGCGCCGGTG',
 'ATCATGTCTT',
 'GTTTGCGTAA',
 'GTGCCGGAGA',
 'AATTCTTAAC',
 'AACGCCGGAT',
 'GGGGTACGAT',
 'TAGTTCGTTT',
 'TTCTGGGTGC',
 'GCAGGGGGAA',
 'GCTATTGTAT',
 'AACAGAGGAA',
 'AATTGGGATG',
 'GTGCCGGAGT',
 'TATTGGACGG',
 'AATTGGGATC',
 'TGCGCCGGTT',
 'GCTATTGTAC',
 'GGGGTACGAG',
 'GCTATTGTAA',
 'GCAGGGGGAT',
 'GCTATTGTAG',
 'GGGGTACGAC',
 'TAGTTCGTTG',
 'ACATTGCTCC',
 'GGCCCTGCGA',
 'GGCCCTGCGC',
 'CGTTACAAAT',
 'ACATTGCTCT',
 'CTAACGTTTC',
 'CGTTACAAAG',
 'GGAATTAAGA',
 'GGAATTAAGC',
 'AGGGAGTATT',
 'TACTCCTTTT',
 'TAAGCTCGTC',
 'CACCAGCGAG',
 'TTCGGGGATC',
 'GAATTCCAGT',
 'TATCACGGAT',
 'TAGGAGCATA',
 'AGGGAGTATG',
 'AGGGAGTA

In [34]:
pickle.dump(umi_list, open('/data/UMI/Optimization/sample_AAAACT_umis.pkl','wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [37]:
report = pickle.load(open('/data/UMI/data/umiReport.pkl','r'))

In [38]:
report

{'AAATTACACG': ([('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570)],
  [],
  [('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570),
   ('chr6', 135016570)]),
 'GATGACATGA': ([('chr10', 120590019),
   ('chr11', 86443288),
   ('chr13', 98309938),
   ('chr13', 98309938),
   ('chr13', 98309938),
   ('chr13', 98309938),
   ('chr1', 27787861),
   ('chr1', 27787863),
   ('chr1', 27787864),
   ('chr1', 27787865),
   ('chr1', 97948684),
   ('chr1', 97948684),
   ('chr1', 97948684),
   ('chr1', 97948684),
   ('chr3'